In [22]:
import os
from twilio.rest import Client
from twilio_config import *
import time

import pandas as pd
import requests

from tqdm import tqdm


In [23]:
query = 'Bogotá'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=295b01273899442c8dd23949232106&q=Bogotá&days=1&aqi=no&alerts=no'

In [24]:
response = requests.get(url_clima).json()
print(response)

{'location': {'name': 'Bogotá', 'region': 'Cundinamarca', 'country': 'Kolumbien', 'lat': 4.6, 'lon': -74.08, 'tz_id': 'America/Bogota', 'localtime_epoch': 1687381980, 'localtime': '2023-06-21 16:13'}, 'current': {'last_updated_epoch': 1687381200, 'last_updated': '2023-06-21 16:00', 'temp_c': 20.0, 'temp_f': 68.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 11.9, 'wind_kph': 19.1, 'wind_degree': 150, 'wind_dir': 'SSE', 'pressure_mb': 1025.0, 'pressure_in': 30.27, 'precip_mm': 0.5, 'precip_in': 0.02, 'humidity': 49, 'cloud': 50, 'feelslike_c': 20.0, 'feelslike_f': 68.0, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 4.0, 'gust_mph': 3.6, 'gust_kph': 5.8}, 'forecast': {'forecastday': [{'date': '2023-06-21', 'date_epoch': 1687305600, 'day': {'maxtemp_c': 17.2, 'maxtemp_f': 63.0, 'mintemp_c': 8.7, 'mintemp_f': 47.7, 'avgtemp_c': 11.5, 'avgtemp_f': 52.7, 'maxwind_mph': 4.3, 'maxwind_kph': 6.8, 'totalprecip_

In [25]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [26]:
# información que se tiene de forcastday
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [27]:

len(response['forecast']['forecastday'][0]['hour'])

24

In [28]:
# pronostico de la 1:00 AM
response['forecast']['forecastday'][0]['hour'][1]

{'time_epoch': 1687327200,
 'time': '2023-06-21 01:00',
 'temp_c': 8.8,
 'temp_f': 47.8,
 'is_day': 0,
 'condition': {'text': 'Partly cloudy',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png',
  'code': 1003},
 'wind_mph': 3.8,
 'wind_kph': 6.1,
 'wind_degree': 112,
 'wind_dir': 'ESE',
 'pressure_mb': 1020.0,
 'pressure_in': 30.11,
 'precip_mm': 0.0,
 'precip_in': 0.0,
 'humidity': 92,
 'cloud': 36,
 'feelslike_c': 8.1,
 'feelslike_f': 46.6,
 'windchill_c': 8.1,
 'windchill_f': 46.6,
 'heatindex_c': 8.8,
 'heatindex_f': 47.8,
 'dewpoint_c': 7.6,
 'dewpoint_f': 45.7,
 'will_it_rain': 0,
 'chance_of_rain': 0,
 'will_it_snow': 0,
 'chance_of_snow': 0,
 'vis_km': 10.0,
 'vis_miles': 6.0,
 'gust_mph': 7.2,
 'gust_kph': 11.5,
 'uv': 1.0}

In [29]:
# Obtener unicamente la fecha del dato anterior
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[0]

'2023-06-21'

In [30]:
# Obtener unicamente la hora del dato anterior
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0])

1

In [31]:
# Obtener la condición del dato anterior
response['forecast']['forecastday'][0]['hour'][1]['condition']['text']

'Partly cloudy'

In [32]:
# Obtener la temperatura del dato anterior
response['forecast']['forecastday'][0]['hour'][1]['temp_c']

8.8

In [33]:
# Obtener si va llover o no (0 o 1) del dato anterior
response['forecast']['forecastday'][0]['hour'][1]['will_it_rain']

0

In [34]:
# Obtener probabilidad de lluvia del dato anterior
response['forecast']['forecastday'][0]['hour'][1]['chance_of_rain']

0

# DataFrames

In [35]:
def get_forecast(response, i):
    date = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hour = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condition = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    temperature = response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']

    return date, hour, condition, temperature, rain, prob_rain


In [36]:
data = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])), colour='blue'):
    data.append(get_forecast(response,i))

100%|██████████| 24/24 [00:00<00:00, 9925.39it/s]


In [37]:
data[0]

('2023-06-21', 0, 'Partly cloudy', 8.9, 0, 0)

In [38]:
columns = ['date', 'hour', 'condition', 'temperature', 'rain', 'prob_rain']
df_forecast = pd.DataFrame(data=data, columns=columns)
df_forecast.head()


,date,hour,condition,temperature,rain,prob_rain
0,2023-06-21,0,Partly cloudy,8.9,0,0
1,2023-06-21,1,Partly cloudy,8.8,0,0
2,2023-06-21,2,Partly cloudy,8.7,0,0
3,2023-06-21,3,Partly cloudy,8.7,0,0
4,2023-06-21,4,Partly cloudy,8.7,0,0


In [39]:
# filtrar los horarios de lluvia entre las 6am a 10pm
df_rain = df_forecast[(df_forecast['rain'] == 1) & (df_forecast['hour']>6) & (df_forecast['hour']< 22)] 
df_rain = df_rain[['hour', 'condition']]
df_rain.set_index('hour', inplace=True)
df_rain

,condition
hour,
10,Patchy light drizzle
12,Patchy rain possible
13,Patchy light rain
14,Light rain shower
16,Patchy rain possible


# Mensaje - Template

In [40]:
msj = "\nHi! \n\n\n Today's forecast is: "+ df_forecast['date'][0] +' in ' + query +' is : \n\n\n ' + str(df_rain)

In [41]:
MY_PHONE_NUMBER

'+57 317 640 8266'

In [42]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body=msj,
                     from_=PHONE_NUMBER,
                     to=MY_PHONE_NUMBER
                 )

print('Mensaje Enviado ' + message.sid)

Mensaje Enviado SM601c169c0e6cf48c9a218fddbe4fb768
